In [1]:
import pandas as pd


In [2]:
excel_path = "../data/4_GEOMAR-relevant_COG_grantees_and_panel_members_2013-2024.xlsx"
sheet_name = "Panel members"
filter_call = "CoG 2024"

In [3]:
df = pd.read_excel(excel_path, sheet_name=sheet_name)
# strip the column names
df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
display(df.head())

# filter
#df = df[df['Call'] == filter_call].reset_index(drop=True)

# Zeige alle Spaltennamen
print("Verfügbare Spalten:")
print(df.columns.tolist())



# Nur wenn die Spalten existieren
if 'First name' in df.columns and 'Last name' in df.columns:
    df['Name'] = df["First name"] + " " + df["Last name"].str.strip()
else:
    print("\n⚠️ Kann 'Name' nicht erstellen - Spalten fehlen!")

display(df.head())

,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024


Verfügbare Spalten:
['Last name', 'First name', 'Host institution (HI)', 'Field', 'Panel', 'Function', 'Domain', 'Call']


,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,Name
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014,Jon Agren
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016,Jon Agren
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014,Francis Albarede
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016,Francis Albarede
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024,Nadia Ameli


In [4]:
# show dataframe where Name is not null and other columnas contain Nan
df_member = df[df['Name'].notnull()].reset_index(drop=True)

empty_fields = df_member[df_member["Field"].isnull()]
display(empty_fields)

unique_members = empty_fields['Name'].unique()
print(f"\nAnzahl der einzigartigen Mitglieder ohne 'Field': {len(unique_members)}")



,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,Name
8,Anderluh,Gregor,NaN,NaN,LS9,Member,LS,CoG 2021,Gregor Anderluh
9,Anderluh,Gregor,NaN,NaN,LS9,Member,LS,CoG 2023,Gregor Anderluh
15,Arndt,Nicholas,NaN,NaN,PE10,Member,PE,CoG 2021,Nicholas Arndt
16,Arndt,Nicholas,NaN,NaN,PE10,Member,PE,CoG 2023,Nicholas Arndt
19,Bai,Yuling,NaN,NaN,LS9,Member,LS,CoG 2021,Yuling Bai
...,...,...,...,...,...,...,...,...,...
543,Virrantaus,Kirsi,NaN,NaN,SH7,Member,SH,CoG 2021,Kirsi Virrantaus
544,Virrantaus,Kirsi,NaN,NaN,SH7,Member,SH,CoG 2023,Kirsi Virrantaus
561,Wu,Xiaogang,NaN,NaN,SH7,Member,SH,CoG 2021,Xiaogang Wu
562,Xu,Chong-Yu,NaN,NaN,PE10,Member,PE,CoG 2021,Chong-Yu Xu



Anzahl der einzigartigen Mitglieder ohne 'Field': 68


In [5]:
from pprint import pprint

pprint(unique_members)
pprint(len(unique_members))

array(['Gregor Anderluh', 'Nicholas Arndt', 'Yuling Bai',
       'Michael Benton', 'Petra Boevink', 'Peter Bols',
       'Kim A. Boutilier', 'Donald E. Canfield', 'Jørgen Carling',
       'Paul Cloke', 'Marcus Collier', 'Pierre Phillippe Combes',
       'Pierre-Philippe Combes', 'Cláudia de Freitas', 'Ben Derudder',
       'Paul Ekins', 'Martin Funk', 'Toni Gabaldón', 'Andrey Ganopolski',
       'Alexandros Gasparatos', 'Ashleigh Griffin', 'Lars-Anders Hansson',
       'Marie E. Herberstein', 'Niels Høiby', 'Robert Holt',
       'Corinna Hoose', 'Dirk Jacobs', 'Adrian Jenkins', 'Nadja Kabisch',
       'Henrik Kaessmann', 'Inger Kappel Schmidt', 'Kristine Kern',
       'Phoebe Koundouri', 'Julie LaRoche', 'Iñigo Lasa Uzcudun',
       'Philippe Lemanceau', 'Edward Lemke', 'Regina Lindborg',
       'Rosaly Lopes', 'Alessandra Magistrato', 'Johanna Mappes',
       'Anders Meibom', 'Andrzej Mizgajski', 'Andrew Mondschein',
       'Michael Obersteiner', 'Spyros Pandis', 'Dario Papale',
     

In [23]:
import ollama 
import time
import requests

def fetch_openalex_id(name:str) -> dict | None:

    search_name = name.replace(" ", "+")
    base_url = f"https://api.openalex.org/authors?filter=display_name.search:{search_name}"
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        results_list = data.get('results', [])
        
        if not results_list:
            print(f"⚠️ Keine Ergebnisse für {name}")
            return None
        
        # Nimm das erste Ergebnis
        first_result = results_list[0]
        pprint(type(first_result))
        pprint(first_result.keys())
        
        # Extrahiere Informationen
        topics = first_result.get('topics', [])

        # 🔹 Sortieren nach 'count' (absteigend)
        sorted_topics = sorted(topics, key=lambda x: x['count'], reverse=True)

        # 🔹 Die größten 4 auswählen
        top_topics = [t['display_name'] for t in sorted_topics[:1]]

        #pprint(top_topics)
        affiliations = first_result.get('affiliations', [])
        x_concepts = first_result.get('x_concepts', [])

        topics_fields_list = [topic['display_name'] for topic in topics]
        #pprint(topics_fields_list)
        
        affiliation = affiliations[0]["institution"]["display_name"] if affiliations else None
        #pprint(affiliation)
        
        x_concepts_fields_list = [concept['display_name'] for concept in x_concepts if concept["score"] >= 90]
        #pprint(x_concepts_fields_list)
        #pprint(x_concepts)
        

        # Speichere die Ergebnisse in einem Dictionary
        results = {
            "Name": name,
            "topics": top_topics,
            "affiliation": affiliation,
            "x_concepts": x_concepts_fields_list
        }
        return results
    else:
        print(f"⚠️ Fehler bei der Anfrage: {response.status_code}")
        return None
    


def process_researcher_profile(results:dict, stream:bool = False, model:str = "deepseek-r1:1.5b") -> str:

    prompt1 = f"""
    You are a scientific summarizer. Based on the following structured information about a researcher, write a concise, natural-sounding academic profile of about 3 sentences. Focus on their main research areas, using fluent academic language similar to a bio in a research paper.
    Do not call them postdoc or similar terms; just focus on their expertise and research themes.
    Input data:
    
    {results}

    Output format:
    A short paragraph (≈3 sentences) describing the person’s expertise and research themes, written in English.
    """

    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt1}],
        stream=True,
    )

    # Antwort puffern und gleichzeitig anzeigen
    response_text = ""
    for chunk in response:
        content = chunk.get("message", {}).get("content", "")
        print(content, end="", flush=True)
        response_text += content

    # # Jetzt hast du alles gesammelt
    # print("\n\n✅ Streaming beendet.")

    return response_text


df_member_info = pd.DataFrame(columns=["name", "affiliation", "topics", "x_concepts", "generated_profile"])

for member in unique_members[:10]:
    results = fetch_openalex_id(member)
    if results:
        print(f"\n\n--- Profil für {member} ---")
        response_text = process_researcher_profile(
            model="deepseek-r1:1.5b",
            results=results,
            stream=True
        )
        #print(f"\nGeneriertes Profil:\n{response_text}")
        time.sleep(2)  # Kurze Pause zwischen den Anfragen
    else:
        print(f"Keine Daten für {member} gefunden.")

    df_member_info = pd.concat([df_member_info, pd.DataFrame([{
        "name": member,
        "affiliation": results.get("affiliation") if results else None,
        "topics": results.get("topics") if results else None,
        "x_concepts": results.get("x_concepts") if results else None,
        "generated_profile": response_text if results else None
    }])], ignore_index=True)

display(df_member_info)

<class 'dict'>
dict_keys(['id', 'orcid', 'display_name', 'display_name_alternatives', 'relevance_score', 'works_count', 'cited_by_count', 'summary_stats', 'ids', 'affiliations', 'last_known_institutions', 'topics', 'topic_share', 'x_concepts', 'counts_by_year', 'works_api_url', 'updated_date', 'created_date'])


--- Profil für Gregor Anderluh ---
Gregor Anderluh is an expert in lipid membrane structure and behavior, with expertise spanning biology, chemistry, biochemistry, and genetics within the National Institute of Chemistry. His work contributes to advancing knowledge in these fields and its applications in real-world problems.<class 'dict'>
dict_keys(['id', 'orcid', 'display_name', 'display_name_alternatives', 'relevance_score', 'works_count', 'cited_by_count', 'summary_stats', 'ids', 'affiliations', 'last_known_institutions', 'topics', 'topic_share', 'x_concepts', 'counts_by_year', 'works_api_url', 'updated_date', 'created_date'])


--- Profil für Nicholas Arndt ---
Nicholas Arnd

,name,affiliation,topics,x_concepts,generated_profile
0,Gregor Anderluh,National Institute of Chemistry,[Lipid Membrane Structure and Behavior],"[Biology, Chemistry, Biochemistry, Genetics]",Gregor Anderluh is an expert in lipid membrane...
1,Nicholas Arndt,Institut des Sciences de la Terre,[Geological and Geochemical Analysis],[],Nicholas Arndt specializes in geological and g...
2,Yuling Bai,Wageningen University & Research,[Plant-Microbe Interactions and Immunity],"[Biology, Genetics, Chemistry, Biochemistry]",Yuling Bai is an expert in plant-microbe inter...
3,Michael Benton,University of Bristol,[Evolution and Paleontology Studies],[],"Michael Benton, a distinguished researcher at ..."
4,Petra Boevink,James Hutton Institute,[Plant-Microbe Interactions and Immunity],"[Biology, Chemistry, Genetics, Biochemistry]",Petra Boevink specializes in understanding pla...
5,Peter Bols,None,None,None,None
6,Kim A. Boutilier,None,None,None,None
7,Donald E. Canfield,University of Southern Denmark,[Paleontology and Stratigraphy of Fossils],[],Donald E. Canfield is an expert in Paleontolog...
8,Jørgen Carling,Peace Research Institute Oslo,[Migration and Labor Dynamics],[],Jørgen Carling is a researcher whose work focu...
9,Paul Cloke,University of Exeter,[Rural development and sustainability],[],"Paul Cloke, a researcher at the University of ..."


In [2]:
from ddgs import DDGS
import requests
from bs4 import BeautifulSoup

# Suche durchführen
query = "Prof Kim A. Boutilier"
results = DDGS().text(query, region='wt-wt', max_results=5)

# Ergebnisse anzeigen
for r in results:
    print(f"\n{r['title']}\n{r['href']}")

# Textinhalt der ersten Seite scrapen
url = list(results)[0]["href"]
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text
soup = BeautifulSoup(html, "html.parser")

text = ' '.join(p.get_text() for p in soup.find_all("p"))
print("\n--- Textauszug ---\n", text[:1000])



dr. KA (Kim) Boutilier - WUR
https://www.wur.nl/en/persons/kim-boutilier.htm

Kim Boutilier's research works | Wageningen University & Research ...
https://www.researchgate.net/scientific-contributions/Kim-Boutilier-20800113

Kim Boutilier - AIChE
https://www.aiche.org/sbe/community/bio/kim-boutilier

Kim Boutilier - Wageningen University & Research
https://research.wur.nl/en/persons/kim-boutilier

Kim Boutilier Inventions, Patents and Patent Applications - Justia ...
https://patents.justia.com/inventor/kim-boutilier

--- Textauszug ---
 Our Research Our research is focussed on understanding the molecular-genetic basis for plant regeneration through embryogenesis. We study different types of in vitro regeneration, including embryogenesis from haploid male and female gametophytes (pollen and embryo sac) and embryogenesis from vegetative tissues (somatic embryogenesis), as well as different types of seed-based embryogenesis systems (zygotic embryogenesis, egg cell parthenogenesis, mater

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Prof Kim A. Boutilier")


In [ ]:


from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

search.invoke("Gregor Anderluh")

'September 11, 2025 - Schulz. Boutilier went to Vancouver Film School in 1996, as well as the University of King\'s College. 3 weeks ago - The EP featured appearances from Pusha T and Kurtis Blow on the songs "Strangers" and "Going Diamond" respectively, as well as song writing credits from the likes of Ryan Tedder (of OneRepublic), Grammy award-winning songwriter Evan Bogart, Cherry Cherry Boom Boom, and producer Doctor Rosen Rosen. The EP received mixed reviews, comparing Boutilier to the likes of Katy Perry, Nicki Minaj, and Gwen Stefani. July 11, 2025 - Paul André Boutilier (born May 3, 1963) is a Canadian former professional ice hockey defenceman who played with several National Hockey League teams in the 1980s. September 22, 2025 - Teahoon Park; Chihyun Park; Byeonggwan Kim; Haejin Shin; Eunkyoung Kim (2013). "Flexible PEDOT electrodes with large thermoelectric power factors to generate electricity by the touch of fingertips". Energy & Environmental Science. 6 (3): 788. doi:10.10

In [11]:
search = DuckDuckGoSearchResults(output_format="list", num_results=5)

search.invoke("Prof Gregor Anderluh")

[{'snippet': 'Es ist bundesweit nicht einheitlich geregelt, welchen Titel ein Juniorprofessor führen soll (Jun.-Prof. oder Prof.) und ob ihm das Promotionsrecht zuerkannt wird.',
  'title': 'Professor – Wikipedia',
  'link': 'https://de.wikipedia.org/wiki/Professor'},
 {'snippet': 'Feb 3, 2025 · Der Titel des Professors beziehungsweise der Professorin (kurz „Prof.“ beziehungsweise „Prof.in“) ist der höchste akademische Titel, der in der Wissenschaft verliehen …',
  'title': 'Professorentitel: Welche gibt es , wer darf sie führen? | academics',
  'link': 'https://www.academics.de/ratgeber/professoren-titel-titelfuehrung'},
 {'snippet': 'höchster akademischer Titel (der einem/einer [habilitierten] Hochschullehrer [in], verdienten Wissenschaftler [in], Künstler [in] o. Ä. verliehen wird) Grammatik ohne Plural Abkürzung Prof. …',
  'title': 'Professor Rechtschreibung, Bedeutung, Definition, Herkunft Duden',
  'link': 'https://www.duden.de/rechtschreibung/Professor'},
 {'snippet': 'Um dies 

In [15]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region="de-de")

search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format="list", num_results=5)

search.invoke("Prof Gregor Anderluh")

[{'snippet': "Mr. Anderluh , Director of Slovenian National institute of chemistry is saying that new ERA is inspiring, promising, however we need to face it's number of ch...",
  'title': 'Prof . dr. Gregor Anderluh - YouTube',
  'link': 'https://www.youtube.com/watch?v=piI4jgS_ORg'},
 {'snippet': 'G Anderluh , J Pungerčar, B Štrukelj, P Maček, F Gubenšek. Biochemical and Biophysical Research Communications 220 (2), 437-442, 1996.',
  'title': 'Gregor Anderluh - Google Akademik',
  'link': 'https://scholar.google.bg/citations?user=58VbXrAAAAAJ&hl=tr'},
 {'snippet': 'Gregor Anderluh . In this issue of Cell Reports Methods, Sadi et al. present a nuclear magnetic resonance approach for quantitative assessment of protein interactions with lipid...',
  'title': 'Gregor ANDERLUH | Laboratory Head | PhD | National Institute of...',
  'link': 'https://www.researchgate.net/profile/Gregor-Anderluh'},
 {'snippet': 'Gregor Anderluh performs multidisciplinary study in Sphingolipid and Lipid raft i